In [32]:
from dplearning_first_part.dplearning_yushu.book_material.ch06.optimizer_compare_naive import optimizer, params
#%matplotlib inline 是 Jupyter Notebook/Jupyter Lab 环境中使用的 IPython 魔法命令，
# 主要作用是将 Matplotlib 绘制的图形直接嵌入到 Notebook 界面中（而不是弹出独立窗口显示）。
%matplotlib inline
import math
import time
import numpy as np
import torch
import sys
sys.path.append("./utils")
from defined_functions import Timer

### 矢量化加速

In [33]:
n = 10000
a = torch.ones([n])
b = torch.ones([n])
c = torch.zeros(n)
timer = Timer()
for i in range(n):
    c[i] = a[i] + b[i]
f'{timer.stop():.5f} sec'

'0.06371 sec'

In [34]:
timer.start()
d = a + b
f'{timer.stop():.5f} sec'

'0.00021 sec'

### 从零实现线性回归

In [5]:
%matplotlib inline
import random
import torch

torch.matmul(x,y)  
用途：支持更高维度张量和广播机制，适用于多种场景：  
二维矩阵相乘（等价于 torch.mm）。  
矩阵与向量相乘（等价于 torch.mv）。  
批量矩阵相乘（如处理三维张量时）。  
自动处理维度扩展（广播）。  
输入要求：  
输入可以是任意维度，但最后两维需符合矩 阵乘法规则。 注意最后两位需要满足矩阵乘法规则

In [6]:

x=torch.randn([2,3,2])
y=torch.randn([1,2,2])
torch.matmul(x,y)

tensor([[[ 0.0987,  0.4689],
         [-0.9510, -2.7583],
         [-0.1594, -0.6459]],

        [[ 2.1192,  6.1775],
         [ 0.7655,  1.8190],
         [-0.5874, -1.4433]]])

从0实现线性回归

In [7]:
import torch

In [8]:
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
# PyTorch 的隐式规则：一维向量在矩阵乘法中根据位置被隐式视为列向量（右乘时）或行向量（左乘时），但结果会被压缩为一维。
# 数学一致性：这种行为与数学中矩阵乘法的列向量约定一致。
# 推荐实践：在涉及多维运算时，显式管理维度（如使用 unsqueeze 或 reshape）可以提高代码可读性并避免错误
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b

    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices= indices[i: min(i + batch_size, num_examples)]
        yield features[batch_indices], labels[batch_indices]

##Python 中的 yield 关键字用于定义生成器函数（generator），
# 它的核心作用是暂停函数的执行并保留当前状态，使得函数可以逐步产生（生成）一系列值，
# 而不是一次性返回所有结果。这种“惰性计算”特性让生成器在处理大数据、流式处理或无限序列时非常高效。

In [9]:

###定义模型
def linear(x,w,b):
    return torch.matmul(x,w) + b

##损失函数
def square_loss(y_hat,y):
    return (y_hat-y.reshape(y_hat.shape))**2/2


def sgd(params, lr, batch_size):
    with torch.no_grad():
        for param in params:
            param -=lr*param.grad/batch_size
            param.grad.zero_()

In [10]:
#初始化权重
w=torch.normal(0,0.01,size=(2,1),requires_grad=True)
b=torch.zeros(1,requires_grad=True)
lr = 0.03
num_epochs = 3
net = linear
loss = square_loss
batch_size=10

for epoch in range(num_epochs):
    for x,y in data_iter(batch_size, features, labels):
        test_x=net(x,w,b)
        test_y=y
        l = loss(net(x,w,b), y)
        l.sum().backward()
        sgd([w,b],lr,batch_size)
    with torch.no_grad():
        tran_l = loss(net(features,w,b), labels)
        print(f'{epoch},loss{tran_l.mean()}')

NameError: name 'batch_size' is not defined

线性回归 pytorch简洁实现

In [11]:
x=torch.ones([2,3])
w=torch.ones([3])
y=torch.matmul(x, w)

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [12]:
from torch.utils import data

In [25]:
##data.TensorDataset是dataset子类 一个快速把特征标签，针对特定场景（张量数据）做了简化封装
def load_array(data_arrays,batch_size,is_train=True):
    dataset=data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset=dataset,batch_size=batch_size,shuffle=is_train)

batch_size=10
data_iter=load_array((features,labels),batch_size)

In [38]:
###定义模型
from torch import nn

In [57]:
# nn.MSELoss 初始化参数
# 参数列表（PyTorch 1.10+ 版本）
# 参数名	类型	默认值	描述
# reduction	str	'mean'	指定损失的计算方式，可选 'none'、'mean' 或 'sum'。
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.sequential=nn.Sequential(
            nn.Linear(2,1)
        )
        self.loss_function = nn.MSELoss()
    def forward(self, x):
        return self.sequential(x)
    def loss(self,x,y):
        x=self.forward(x)
        return self.loss_function(x,y)
    def optim(self,lr):
        return torch.optim.SGD(self.parameters(),lr=lr)

In [58]:
net=Net()
###参数初始化，使用sequential访问图层，然后weight.data和bias.data方法访问参数
## 可以使用替换方法normal_和fill_来重写参数值。
net.sequential[0].weight.data.normal_(0,0.01)
net.sequential[0].bias.data.fill_(0)

tensor([0.])

In [65]:
epoch=3
for i in range(epoch):
    for x,y in data_iter:
        net.optim(lr=0.03).zero_grad()
        l=net.loss(x,y)
        l.backward()
        net.optim(lr=0.03).step()
    with torch.no_grad():
        l=net.loss(features,labels).mean()
        print(f'{i},loss{l.item()}')

0,loss0.00010285895405104384
1,loss0.00010266688332194462
2,loss0.00010257143730996177


In [63]:
for i in net.parameters():
    print(i)

Parameter containing:
tensor([[ 1.9944, -3.3877]], requires_grad=True)
Parameter containing:
tensor([4.1873], requires_grad=True)
